# Utilidades para obtener caracteristicas del tablero, acomode optimo de piezas, obtener orientaciones, etc.

In [ ]:
import itertools
import numpy as np
from shapely.geometry import Polygon, MultiPolygon, Point, MultiPoint
from shapely.affinity import translate, rotate, scale
from functools import partial
from multiprocessing import Pool

import warnings
warnings.filterwarnings('ignore')

# Funciones para obtener caracteristicas de las piezas del tablero


def crear_poligonos(coordenadas_tablero):
    """
    Crea poligonos con la clase shapely de python usando los puntos centrales
    donde estos puntos centrales consisten en una coleccion de tuplas(x, y ,m) donde 
    x e y son coordenadas del tablero, y la m es un valor entre 0 y 1
    :param coordenadas_tablero: coleccion de puntos del tablero
    :return: arreglo con los poligonos iniciales
    """
    poligonos = []
    for idx, block in enumerate(coordenadas_tablero):
        puntos = np.array(block)[:, :-1]
        poligonos = MultiPoint(puntos+0.5).buffer(0.5, cap_style=3)
        poligonos.checkers = np.array(block)
        poligonos.orientaciones = obtener_orientaciones(poligonos)
        poligonos.id = idx
        poligonos.append(poligonos)
        print(str(np.array(poligonos)))  # Verificamos los poligonos generados
    return np.array(poligonos)


def obtener_orientaciones(poligono):
    """
    Obtiene las posibles rotaciones y caras de una pieza, toma en cuenta la simetria
    y obtiene las orientaciones unicas para el poligono dado
    :param poligono: poligono a operar
    :return: arreglo de orientaciones unicas
    """
    cx, cy = poligono.centroid.x, poligono.centroid.y
    puntos = np.array(poligono.checkers)[:, :-1]
    checkers = np.array(poligono.checkers)[:, -1]
    puntos = puntos[checkers == 1]
    poligono = poligono.difference(MultiPoint(puntos + 0.5).buffer(0.2))

    rotaciones = [0, 90, 180, 270]
    caras = [1, -1]  # 1, cara normal, -1 espejo
    iteraciones = [rotaciones, caras]
    configuracion = np.array(list(itertools.product(*iteraciones)))
    poligonos_unicos = []
    configuracion_unica = []
    for rotacion, vuelta in configuracion:
        p = poligono
        p = rotate(p, rotacion, origin=[cx, cy])
        p = scale(p, vuelta, origin=[cx, cy, 0])
        if np.any([p.difference(u).area < 1e-5 for u in poligonos_unicos]):
            continue
        poligonos_unicos.append(p)
        configuracion_unica.append([rotacion, vuelta])
    return configuracion_unica


def transformacion(poligono, vector_t, angulo_r, cara_f):
    """
    Aplica una transformacion al poligono
    :param poligono: Poligono a ser transformado
    :param vector_t: Vector de traslacion
    :param angulo_r: angulo de rotacion en grados
    :param cara_f: cara de la figura (1,-1)
    :return: poligono transformado
    """
    # Metodos de shapely para operar poligonos
    poly = rotate(poligono, angulo_r, origin=np.array([0.5, 0.5]))
    poly = scale(poligono, cara_f, origin=np.array([0.5, 0.5, 0.0]))
    poly = translate(poligono, *vector_t)
    return poly


def calcular_linea_ext(perfil_tablero):
    """
    Calcula las lineas exteriores e interiores del tablero
    :param profile: perfil_tablero
    :return: perimetro de la figura
    """
    if perfil_tablero is None:
        return np.nan

    outline = perfil_tablero.exterior.length
    for interior in perfil_tablero.interiors:
        outline += interior.length
    return outline


def obtener_nuevo_perfil(perfil, poligono, configuracion):
    """
    Funcion que retorna el perimetro de una figura en particular con una configuracion particular (rotacion, cara)
    colocadas en un tablero generado. Si el poligono no encaja en el perfil o sl el poligono
    divide al perfil en 2 partes, una tupla de (np.nan, None) se retorna
    de otra forma se retorna una tupla con el (perimetro y el nuevo perfil)
    :param perfil: Perfil inicial
    :param poligono: poligono de entrada
    :param configuracion: transformacion a ser aplicada (casillas del tablero, angulo_r, cara_f)
    :return: tupla con (perimetro del nuevo perfil y perfil nuevo con la figura insertada)
    """
    # Poligono temporal donde toma el poligono con sus transformaciones correspondientes
    poligono = transformacion(poligono, *configuracion)
    if not poligono.within(perfil):
        return None

    nuevo_perfil = perfil.difference(poligono)
    if type(nuevo_perfil) == MultiPolygon:
        return None

    return nuevo_perfil


def obtener_perimetro_perfil(perfil, poligono, configuracion):
    """
    Funcion que contiene el calculo del perimetro de un nuevo perfil
    :param perfil: perfil inicial
    :param poligono: poligono a colocar
    :param configuracion: configuraciones para transformar el poligono
    :return: perimetro del nuevo perfil
    """
    return calcular_linea_ext(obtener_nuevo_perfil(perfil, poligono, configuracion))


def colocacion_optima(perfil, poligono, tamanio_tablero, origin_checker=None):
    """
    Funcion para encontrar la colocacion optima de un poligono en un perfil. Si el poligono no puede ser colocado
    en el perfil dado, regresa None, de otra forma se retorna la configuracion optima y el perfil correspondiente.
    :param perfil: perfil de entrada
    :param poligono: poligono a verificar su colocacion optima
    :param tamanio_tablero: tamaño del tablero
    :param origin_checker: checador del origen
    :return: configuracion de la colocacion optima y el perfil correspondiente
    """
    # establece la cuadriula del tablero 8x8=64
    puntos_tablero = np.array([Point([x + 0.5, y + 0.5]) for x in range(tamanio_tablero) for y in range(tamanio_tablero)],
                              dtype=object)
    posicion_pieza = np.array([[x, y] for x in range(tamanio_tablero)
                               for y in range(tamanio_tablero)], dtype=int)
    mascara_puntos = np.array([pt.within(perfil)
                               for pt in puntos_tablero], dtype=bool)

    if origin_checker is not None:
        if origin_checker == poligono.checkers[0, -1]:
            mascara_puntos &= np.sum(posicion_pieza, axis=-1) % 2 == 0
        else:
            mascara_puntos &= np.sum(posicion_pieza, axis=-1) % 2 == 1

    rflips = poligono.orientaciones
    pos = posicion_pieza[mascara_puntos]

    iterables = [pos, rflips]
    configuraciones = list(itertools.product(*iterables))
    configuraciones = [(s[0], *s[1]) for s in configuraciones]

    func = partial(obtener_nuevo_perfil, perfil, poligono)

    pool = Pool(processes=None)
    lineas_exteriores = np.array(pool.map(func, configuraciones))
    pool.close()

    outlines = outlines.astype(float)
    opt_configuraciones = configuraciones[np.argsort(outlines)[0]]
    opt_perfil = obtener_nuevo_perfil(perfil, poligono, opt_configuraciones)

    if opt_perfil is not None:
        return opt_configuraciones, opt_perfil


def funcion_cantor(a, b):
    """
    Funcion de cantor usada para obtener numeros unicos de un par de numeros
    :param a: numero de entrada
    :param b: otro numero de entrada
    :return: numero operado
    """
    return 0.5 * (a + b + 1) * (a + b) + b


def obtener_config_optima(cromosoma, poligonos_iniciales, tamanio_tablero):
    """
    Put each piece in the chromosome order in the optimal place until the final configuration is
    achieved. return the fitness level and the corresponding placements.
    Coloca cada pieza en el orden optimo dentro del cromosoma hasta que se llena todo el cromosoma
    retornando la aptitud y los elementos correspondientes
    :param cromosoma: cromosoma a ser llenado
    :param poligonos iniciales: lista de poligonos inicial
    :param tamanio_tablero: tamaño del tablero
    :return: colocaciones y aptitud
    """
    perfil = Polygon(
        [[0, 0], [tamanio_tablero, 0], [tamanio_tablero, tamanio_tablero], [0, tamanio_tablero]])
    colocaciones = np.full_like(cromosoma, None, dtype=object)
    origin_checker = None
    for idx, c in enumerate(cromosoma):
        p = poligonos_iniciales[c]
        try:
            opt, perfil = colocacion_optima(
                perfil, p, tamanio_tablero, origin_checker)
            colocaciones[idx] = opt
            if origin_checker is None:
                sum_vec = np.sum(opt[0])
                if sum_vec % 2 == p.checkers[0, -1]:
                    origin_checker = 0
                else:
                    origin_checker = 1
        except Exception:
            idx -= 1
            break
    area_vacia = perfil.area
    piezas_sin_uso = len(cromosoma) - idx - 1
    linea_exterior = calcular_linea_ext(perfil)
    f_aptitud = area_vacia + piezas_sin_uso + linea_exterior
    return colocaciones, f_aptitud


### Metodos usados para graficar tanto el cromosoma como la grafica del historial de evolucion

In [ ]:
import numpy as np
import geopandas as gpd
import matplotlib.pyplot as plt

from shapely.geometry import MultiPoint, Polygon

def graficar_cromosoma(individuo, poligonos, tamanio_tablero, nom_archivo=''):
    """
    Funcion para graficar el tablero con las piezas ensambladas
    :param individuo: El individuo que contiene todos los atributos como la funcion de aptitud
    :param poligonos: lista de poligonos
    :param tamanio_tablero: tamanio de nuestro tablero
    :param nom_archivo: nombre del fichero a guardar
    """
    nuevos_poligonos = []
    for poligono in poligonos:
        puntos = np.array(poligono.checkers)[:, :-1]
        checkers = np.array(poligono.checkers)[:, -1]
        puntos = puntos[checkers == 1]
        poligono_d = poligono.difference(MultiPoint(puntos + 0.5).buffer(0.25))
        nuevos_poligonos.append(poligono_d)

    nuevos_poligonos = np.array(nuevos_poligonos)
    nuevos_poligonos = [transform(p, *s) for p, s in zip(nuevos_poligonos[individuo.cromosoma],
                                                         individuo.colocaciones) if s is not None]
    indices = [individuo.cromosoma[idx]
               for idx, s in enumerate(individuo.colocaciones) if s is not None]
    canvas = Polygon([[0, 0], [tamanio_tablero, 0], [
                     tamanio_tablero, tamanio_tablero], [0, tamanio_tablero]])
    gdf = gpd.GeoDataFrame({'idx': indices}, geometry=nuevos_poligonos)

    fig, ax = plt.subplots(1, 1, figsize=(10, 10))
    ax.plot(*canvas.exterior.xy, 'k')
    gdf.plot(ax=ax, column='idx', edgecolor='black', alpha=0.75,
             vmin=0, vmax=len(poligonos), cmap=plt.cm.hot)
    ax.set_ylim([-0.1, tamanio_tablero + 0.1])
    ax.set_xlim([-0.1, tamanio_tablero + 0.1])
    ax.set_title(f'[{",".join(individuo.cromosoma.astype(str))}]',
                 fontsize=20, fontname='serif')
    ax.set_xlabel(
        f'Aptitud: {int(individuo.f_aptitud)}', fontsize=20, fontname='serif')
    ax.set_xticks([])
    ax.set_yticks([])

    if nom_archivo != '':
        fig.savefig(nom_archivo, dpi=300, bbox_inches='tight')
        plt.close()


def graficar_historial(historial, nom_archivo=''):
    """
    Funcion para graficar el proceso de evolucion
    :param historial: datos para cada generacion (historial)
    :param nom_archivo: nombre del archivo a guardar 
    """
    fig, ax = plt.subplots(1, 1, figsize=(10, 10))

    for idx, gen in enumerate(historial):
        ax.scatter(np.full_like(gen, idx), gen, s=50, c='k', marker='o')
        ax.set_title('Proceso de evolucion', fontsize=20, fontname='serif')
        ax.set_xlabel('Numero de generacion', fontsize=20, fontname='serif')
        ax.set_ylabel('Aptitud', fontsize=20, fontname='serif')
        ax.set_ylim(bottom=0.0)

    plt.plot(range(len(historial)), np.mean(
        historial, axis=1), label='Valor de la generacion')
    plt.fill_between(range(len(historial)), np.mean(historial, axis=1) - np.std(historial, axis=1),
                     np.mean(historial, axis=1) + np.std(historial, axis=1),
                     alpha=0.5, interpolate=True)
    plt.legend(prop={'family': 'serif', 'size': 15})

    if nom_archivo != '':
        fig.savefig(nom_archivo, dpi=300, bbox_inches='tight')
        plt.close()


### Algoritmo genetico

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from tqdm.auto import tqdm
l
class Individuo(object):
    """
    Esta clase representa al individuo
    """

    def __init__(self, poligonos, tamanio_tablero, cromosoma=None):
        """
        Un nuevo cromosoma es generado si no se especifica uno
        :param poligonos: lista de poligonos
        :param tamanio_tablero: tamaño del tablero
        :param cromosoma: establece el orden en que se colocaran los poligonos
        """
        self.n_piezas = len(poligonos)  # Numero de piezas del tablero
        # Genera un nuevo cromosoma, si se especifica uno se obtieien sus valores por medio de un arreglo
        self.cromosoma = self.generar_cromosoma(
        ) if cromosoma is None else np.array(cromosoma)
        self.poligonos = poligonos  # Lista de poligonos
        self.tamanio_tablero = tamanio_tablero  # Tamaño del tablero
        self.colocaciones, self.f_aptitud = obtener_config_optima(
            self.cromosoma, poligonos, tamanio_tablero)  # Generamos las colocadas y la funcion de aptitud

    def generar_cromosoma(self):
        """
        Crea una secuencia aleatoria de numeros para el cromosoma
        """
        return np.random.choice(range(self.n_pieces), self.n_pieces, replace=False).astype(int)

    def mutacion(self, probabilidad_mutacion):
        """
        Muta al cromosoma intercambiando el gen con un gen aleatorio si se cumple la condicion de cruza
        :param probabilidad_mutacion: probabilidad de que la mutacion ocurra
        """
        cromosoma = self.cromosoma.copy()  # Copiamos el cromosoma
        ha_mutado = False
        for idx, c in enumerate(cromosoma):
            if np.random.rand() < probabilidad_mutacion:
                ha_mutado = True
                # Seleccionamos una posicion aleatoria del cromosoma
                rand_idx = np.random.choice(range(len(cromosoma)))
                # hacemos mutacion por medio de intercambio de genes
                cromosoma[idx], cromosoma[rand_idx] = cromosoma[rand_idx], cromosoma[idx]

        if ha_mutado:
            self.cromosoma = cromosoma  # Actualizamos el cromosoma mutado
            self.colocaciones, self.f_aptitud = obtener_config_optima(
                self.cromosoma, self.poligonos, self.tamanio_tablero)  # Actualizamos las colocaciones y la funcion de aptitud

    def cruza(self, acompanante):
        """
        Establecemos la cruza por medio de ox
        :param acompanante: el segundo individuo con el que se cruzara
        :return: dos nuevos hijos que son resultado de la cruza
        """
        c1, c2 = np.zeros((2, self.n_piezas), dtype=int)
        inicio, fin = np.sort(np.random.choice(
            range(self.n_piezass), 2, replace=False))

        p1 = self.cromosoma
        p2 = acompanante.cromosoma

        c1[inicio:fin + 1] = p1[inicio:fin + 1]
        mascara1 = ~np.in1d(p1, p1[inicio:fin + 1])
        mascara2 = ~np.in1d(p2, p1[inicio:fin + 1])
        c1[mascara1] = p2[mascara2]

        c2[inicio:fin + 1] = p2[inicio:fin + 1]
        mascara1 = ~np.in1d(p2, p2[inicio:fin + 1])
        mascara2 = ~np.in1d(p1, p2[inicio:fin + 1])
        c2[mascara1] = p1[mascara2]

        return Individuo(self.poligonos, self.tamanio_tablero, c1), Individuo(self.poligonos, self.tamanio_tablero, c2)

    def grafica(self, nom_archivo=''):
        """
        Grafica el tablero con las piezas colocadas
        :param nom_archivo: nombre del archivo donde se guardara la imagen
        """
        if nom_archivo == '':
            graficar_cromosoma(self, self.poligonos, self.tamanio_tablero)
        else:
            graficar_cromosoma(self, self.poligonos,
                               self.tamanio_tablero, nom_archivo)


class Evolucion(object):
    """
    Clase que representa la evolucion de la poblacion
    """

    def __init__(self, numero_poblacion, poligonos, tamanio_tablero, probabilidad_mutacion=0.0):
        """
        Funcion que inicializa la poblacion, un cromosoma aleatorio es generado si no se especifica uno
        :param num_poblacion: tamanio de la poblacion para trabajar
        :param polygons: lista de figuras inicial
        :param board_size: tamanio del tablero
        :param mutation_probability: probabilidad de que ocurra una mutacion
        """
        self.num_poblacion = numero_poblacion + \
            1 if numero_poblacion % 2 != 0 else numero_poblacion
        self.poligonos = poligonos
        self.tamanio_tablero = tamanio_tablero
        self.probabilidad_mutacion = probabilidad_mutacion
        self.poblacion = None
        self.mejor = None
        self.historial = []

    def inicializar_poblacion(self):
        """
        inicializamos una nueva poblacion
        :return: una lista de individuos
        """
        print(f'Generacion: {len(self.history)}: Inicializando')
        self.poblacion = [Individuo(self.poligonos, self.tamanio_tablero) for _ in tqdm(range(self.num_poblacion),
                                                                                        leave=False)]
        self.mutar_generacion()

    def seleccion(self):
        """
        Selecciona dos individuos de la poblacion aleatoriamente, donde la probabilidad de elegir un individuo
        esta ponderada por su nivel de aptitud.
        :return: 2 individuos seleccionados
        """
        # Obtenemos la aptitud de cada individuo
        lista_aptitudes = np.array([ind.f_aptitud for ind in self.poblacion])
        lista_aptitudes = -lista_aptitudes + \
            np.min(lista_aptitudes) + np.max(lista_aptitudes) + 1
        lista_aptitudes = lista_aptitudes / np.sum(lista_aptitudes)
        p1, p2 = np.random.choice(self.poblacion,
                                  size=2,
                                  replace=False,
                                  p=lista_aptitudes)
        return p1, p2

    def mutar_generacion(self):
        """
        Aplica una ronda de mutacion a la poblacion
        """
        for ind in self.poblacion:
            ind.mutacion(self.probabilidad_mutacion)

    def siguiente_generacion(self):
        """
        Genera una nueva generacion de descendientes
        """
        print(f'Generacion: {len(self.historial)}')
        nueva_poblacion = []
        for _ in tqdm(range(int(self.num_poblacion/2)), leave=False):
            p1, p2 = self.seleccion()
            c1, c2 = p1.mutacion(p2)
            nueva_poblacion.extend([c1, c2])
        self.poblacion = nueva_poblacion
        self.mutar_generacion()

    def verificar_parada(self):
        """
        Checa si se ha alcanzado el numero esperado de generaciones
        :return: boolean
        """
        lista_aptitud = np.array(
            [ind.f_aptitud for ind in self.poblacion], dtype=int)
        if np.any(lista_aptitud == 0):
            return True

    def obtener_mejor_candidato(self):
        """
        Obtiene el mejor individuo de la poblacion actual
        :return: el mejor individuo
        """
        mejor_id = np.argsort([ind.f_aptitud for ind in self.poblacion])[0]
        return self.poblacion[mejor_id]

    def graficar_proceso(self, nom_archivo=''):
        """
        Grafica el proceso de evolucion de la poblacion
        :param filename: filename for saving the image
        """
        if nom_archivo == '':
            graficar_historial(np.array(self.history))
        else:
            graficar_historial(np.array(self.historial), nom_archivo)

    def ejecutar(self, nom_archivo=''):
        """
        Ejecuta el proceso de evolucion
        :param nom_archivo: nombre del archivo
        """
        self.inicializar_poblacion()  # Primero inicializamos la poblacion
        # De la poblacion generada obtenemos el mejor candidato
        self.mejor = self.obtener_mejor_candidato()
        lista_aptitudes = np.array(
            [ind.f_aptitud for ind in self.poblacion], dtype=int)  # Añadimos las aptitudes de los demas indiviuos a una lista
        # Dentro de arreglo que hace la funcion de historial guardamos nuestras aptitudes
        self.historial.append(lista_aptitudes)
        # Graficamos el proceso usando el historial de aptitudes
        self.graficar_proceso(nom_archivo)
        plt.close()
        # Sacamos el mejor candidato guardado en la variable "mejor"
        # Mostramos su aptitu y su cromosoma de este mejor candidato
        print(f'Aptitud del mejor candidato: {self.mejor.f_aptitud},'
              f'        cromosoma: [{",".join(np.array(self.mejor.cromosoma, dtype=str))}]')
        # Mientras no se encuentre la mejor solucion, se hara un ciclo
        while not self.verificar_parada():
            # Generamos una nueva poblacion
            self.siguiente_generacion()
            # Obtenemos al mejor candidato de esa generacion
            mejor = self.obtener_mejor_candidato()
            # Sacamos el resto de aptitudes de los demas candidatos y las guardamos en un arreglo
            lista_aptitudes = np.array(
                [ind.f_aptitud for ind in self.poblacion], dtype=int)
            # Al historial de aptitudes, añadimos la lista de aptitudes anterior
            self.historial.append(lista_aptitudes)
            # Graficamos el proceso de las aptitudes generadas en todas las poblaciones
            self.graficar_proceso(nom_archivo)
            plt.close()
            # Si la aptitud del individuo de la generacion actual es mejor a la del individuo anterior
            # pasa a ser la nueva mejor aptitud y se guarda dentro de la variable mejor
            if mejor.f_aptitud < self.mejor.f_aptitud:
                self.mejor = mejor
            print(f'Aptitud del mejor candidato: {self.mejor.f_aptitud}, '
                  f'cromosoma: [{",".join(np.array(self.mejor.cromosoma, dtype=str))}]')


### Ejecutar algoritmo

In [ ]:
import numpy as np
import matplotlib

from genetic_algo import Evolucion
from utils import *

matplotlib.use('Agg')

# Coordenada en x --- ,  y | , dama 0 no, 1 si
coordenadas_piezas = np.array([
    [(0, 0, 0), (1, 0, 1), (2, 0, 0), (3, 0, 1), (4, 0, 0)], # ID = 0
    [(0, 0, 0), (1, 0, 1), (2, 0, 0), (2, -1, 1), (1, 1, 0)], # ID = 1
    [(0, 0, 0), (1, 0, 1), (2, 0, 0), (0, 1, 1), (2, -1, 1)], # ID = 2
    [(0, 0, 0), (1, 0, 1), (2, 0, 0), (-1, 0, 1), (0, -1, 1)], # ID = 3
    [(0, 0, 0), (-1, 0, 1), (1, 0, 1), (0, 1, 1), (0, -1, 1)], # ID = 4
    [(0, 0, 1), (1, 0, 0), (2, 0, 1), (0, 1, 0), (0, -1, 0)], # ID = 5
    [(0, 0, 0), (1, 0, 1), (2, 0, 0), (3, 0, 1), (0, 1, 1)], # ID = 6
    [(0, 0, 0), (0, 1, 1), (2, 0, 0), (2, 1, 1), (1, 1, 0)], # ID = 7
    [(0, 0, 1), (1, 0, 0), (2, 0, 1), (0, 1, 0), (0, 2, 1)], # ID = 8
    [(0, 0, 1), (-1, 0, 0), (0, 1, 0), (1, 1, 1), (2, 1, 0)], # ID = 9
    [(0, 0, 0), (1, 0, 1), (0, 1, 1), (1, 1, 0), (-1, 1, 0)], # ID = 10
    [(0, 0, 1), (1, 0, 0), (0, 1, 0), (1, 1, 1)], # ID = 11
    [(0, 0, 0), (1, 1, 0), (2, 2, 0), (0, 1, 1), (1, 2, 1)] # ID = 12
])

tamanio_tablero = 8
poligonos = crear_poligonos(coordenadas_piezas)

evo = Evolucion(50, poligonos, tamanio_tablero, probabilidad_mutacion=0.01)

evo.ejecutar('./evolucion.png')
evo.mejor.grafica('./solucion_optima.png')